In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import pandas as pd
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt

# Customize plot settings
from latexify import *
from utils import *
golden_mean = (np.sqrt(5) - 1.0) / 2.0  # Aesthetic ratio
latexify(fig_width=5)

In [2]:
load_data = pd.read_csv('data/loads.csv', parse_dates=[0], index_col=0)["Load (kW)"]
load_min, load_max = load_data.min(), load_data.max()
tou_price_data = pd.read_csv('data/tou_prices.csv', parse_dates=[0], index_col=0)["TOU Price (NOK/kWh)"]
spot_price_data = pd.read_csv('data/spot_prices.csv', parse_dates=[0], index_col=0)["Spot Price (NOK/kWh)"]

# Load forecasting
load_baseline = pd.read_csv('data/load_baseline.csv', parse_dates=[0], index_col=0)["Baseline load (kW)"]
AR_params = pd.read_pickle("data/AR_params.pickle")
sigma_residual_errors = pd.read_pickle("data/sigma_residual_errors.pickle")

# Define simulation window: one year (2022) with hourly resolution
T = 24 * 365
start_date = pd.Timestamp("2022-01-01 00:00:00", tz=None)

sim_start_time = load_baseline.index.get_loc(start_date)
sim_end_time = sim_start_time + T
dt = load_data.index[sim_start_time:sim_end_time]

# Load and price values for 2022
l = load_data.iloc[sim_start_time:sim_end_time].values
tou_prices = tou_price_data.iloc[sim_start_time:sim_end_time].values
spot_prices = spot_price_data.iloc[sim_start_time:sim_end_time].values

K, L = 12, 5

## Prescient problem

In [12]:
# Constants and parameters definition
P, N, Q = 20, 3, 40
q_init, q_final, C, D = Q/2, Q/2, Q/2, Q/2
eta_s, eta_c, eta_d = 0.99998, 0.95, 0.95 
tier_costs = np.array([83, 147, 252, 371, 490])
tier_thresholds = np.array([2, 5, 10, 15])
T, K, L = len(l)+1, len(set(dt.month)), len(tier_costs)

print(T, K, L)

# # Variables
# p = cp.Variable(T-1, nonneg=True)  # Power from grid
# c = cp.Variable(T-1, nonneg=True)  # Charging power
# d = cp.Variable(T-1, nonneg=True)  # Discharging power
# q = cp.Variable(T, nonneg=True)  # Energy in storage
# s = cp.Variable((K, L), boolean=True)  # Binary variables for tiers

# # Constraints
# cons = [
#     p <= P,  # Power from grid constraint
#     p + d - l - c == 0,  # Energy balance constraint
#     q[1:] == eta_s * q[:-1] + eta_c * c - (1/eta_d) * d,  # Storage dynamics constraint
#     q[0] == q_init,  # Initial storage level
#     q[-1] == q_final,  # Final storage level
#     q <= Q,  # Storage capacity constraint
#     c <= C,  # Charging capacity constraint
#     d <= D  # Discharging capacity constraint
# ]

# # Energy usage charge
# energy_cost = cp.sum(cp.multiply(tou_prices + spot_prices, p))

# # Peak power charges
# peak_power_cost = 0
# for k in range(K):  # Iterate over months
#     # Compute daily max power for each day of the month
#     m_k = [cp.max(p[(dt.date == day) & (dt.month == k+1)]) 
#            for day in pd.unique(dt[dt.month == k+1].date)]
#     # Compute average of N largest daily max powers
#     z_k = cp.sum_largest(cp.hstack(m_k), N) / N
#     # Add to peak power cost
#     peak_power_cost += cp.matmul(tier_costs, s[k, :])
#     # Add constraints for tier thresholds and total number of tiers
#     cons += [z_k <= cp.matmul(tier_thresholds, s[k, :-1]) + P * s[k, -1], cp.sum(s[k, :]) == 1]

# # Problem definition and solving
# problem = cp.Problem(cp.Minimize(energy_cost + peak_power_cost), cons)  # Problem definition
# problem.solve()  # Solve problem using Gurobi solver


8761 12 5
